# [Title]

## 1. Introduction

### 1.1 Literature review of the compound (<500 words)

### 1.2 Literature review of the cell factory (<500 words)

## 2. Problem definition (<300 words)

## 3. Selection and assessment of existing GSM (<500 words)

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project)

### 4.1 Importing the necessary modules and loading the model

In [1]:
import cobra
from cobra.io import read_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra.medium import minimal_medium

In [2]:
model = read_sbml_model('models/Sco-GEM.xml')

### 4.2 Flux Balance analysis of the initial model

In [3]:
medium = model.medium
medium

{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 0.8,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [4]:
sol = model.optimize()
print(f'The objective is to {model.objective}')

The objective is to Maximize
1.0*BIOMASS_SCO_tRNA - 1.0*BIOMASS_SCO_tRNA_reverse_d3620


In [5]:
print(f'Biomass production rate is {sol.fluxes.BIOMASS_SCO_tRNA:.3f} mmol/gDRW/h.')
print(f'The production of antibiotic is {sol.fluxes.DM_RED_c} mmol/gDRW/h.')

Biomass production rate is 0.075 mmol/gDRW/h
The production of antibiotic is 0.0 mmol/gDRW/h


### 4.3 Flux balance analysis for the maximization of antibiotic production

In [6]:
with model:
    model.objective = model.reactions.DM_RED_c
    sol = model.optimize()
    print(f'The objective is to {model.objective}')
    print('')
    print(f'Biomass production rate is {sol.fluxes.BIOMASS_SCO_tRNA} mmol/gDRW/h.')
    print(f'The production of antibiotic is {sol.fluxes.DM_RED_c:.3f} mmol/gDRW/h.')

The objective is to Maximize
1.0*DM_RED_c - 1.0*DM_RED_c_reverse_13383

Biomass production rate is 0.0 mmol/gDRW/h
The production of antibiotic is 0.132 mmol/gDRW/h


### 4.4 Medium optimization

#### Anaerobic conditions

In [7]:
with model:
    medium = model.medium
    medium["EX_o2_e"] = 0.0
    model.medium = medium
    solution = model.optimize()

    print("\nAnaereobic conditions")
    print("Biomass production:", model.slim_optimize())


Anaereobic conditions
Biomass production: 0.0


#### Minimal medium

In [8]:
minimal_medium(model,model.slim_optimize())

EX_ca2_e        0.000429
EX_cl_e         0.000429
EX_cobalt2_e    0.000003
EX_cu2_e        0.000058
EX_fe2_e        0.001453
EX_glc__D_e     0.800000
EX_k_e          0.016078
EX_mg2_e        0.000715
EX_mn2_e        0.000057
EX_mobd_e       0.000003
EX_nh4_e        0.651820
EX_ni2_e        0.000027
EX_o2_e         1.727388
EX_pi_e         0.073858
EX_so4_e        0.019368
EX_zn2_e        0.000028
dtype: float64

#### Carbon sources

In [19]:
carbon_sources = []

for reaction in model.exchanges:
    for metabolite in reaction.metabolites:
        if 'C' in metabolite.elements:
            carbon_sources.append(reaction)

with model:
    model.objective = model.reactions.DM_RED_c
    threshold = model.slim_optimize(error_value=0.0)

solution_dict = {}

for cs in carbon_sources:
    with model:
        medium = model.medium
        medium['EX_glc__D_e'] = 0 # remove glucose from medium
        medium[cs.id] = 0.8 # add carbon source to medium
        model.medium = medium
        model.objective = model.reactions.DM_RED_c
        solution = model.slim_optimize(error_value=0.0)

        if solution > threshold:
            solution_dict[cs.id] = solution

print(f'The carbon sources that produce better results than glucose given that their uptake flux is 0.8 mmol/gDRW/h are:')
print('')
print(f'{"Carbon source":^42} | Production of antibiotic [mmol/gDW/h]')
print('-'*82)
for source, production in sorted(solution_dict.items(), key=lambda item: item[1], reverse=True):
    print(f'{list(model.reactions.get_by_id(source).metabolites)[0].name:>42} | {production:>6.3f}')

The carbon sources that produce better results than glucose given that their uptake flux is 0.8 mmol/gDRW/h are:

              Carbon source                | Production of antibiotic [mmol/gDW/h]
----------------------------------------------------------------------------------
Cellulose (chain length 500 glc_D assumed) | 72.603
                         Xylan (Oat spelt) | 64.308
                             Maltoheptaose |  1.027
                              Maltohexaose |  0.880
                        1,4-alpha-D-glucan |  0.880
                             Maltopentaose |  0.727
                             Maltotetraose |  0.580
                               Maltotriose |  0.432
                                 Raffinose |  0.426
                   N,N'-diacetylchitobiose |  0.363
                                Melibiitol |  0.289
                                 Trehalose |  0.284
                                   Maltose |  0.284
                                 Melibiose |

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References